In [1]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import folium
import openrouteservice as ors
from geopy.distance import great_circle

In [2]:
def get_token():
    load_dotenv('./.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    client_id = os.environ.get("X-ClientId")
    pass_key = os.environ.get("passKey")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [3]:
get_token()

b'{"code": "01", "description": "Token extend  into control-cache Data recovered  OK, (lapsed: 205 millsecs)", "datetime": "2024-02-02T14:09:52.182444", "data": [{"XClientId": "09141f1a-bf02-4e23-80ee-e7fa7ea819d2", "nameApp": "OPENAPI MobilityLabs", "updatedAt": "2024-01-09T17:02:16.9230000", "userName": "silviluliuma", "lastUpdate": {"$date": 1706879279102}, "idUser": "26ddfea8-f2ce-438f-89e4-5110c9abd07e", "priv": "U", "tokenSecExpiration": 86399, "email": "valeromsilvia@gmail.com", "tokenDteExpiration": {"$date": 1706969278102}, "flagAdvise": true, "accessToken": "70d9c557-201e-41c1-a674-f0e8953d0d22", "apiCounter": {"current": 14, "dailyUse": 20000, "owner": 0, "licenceUse": "Please mention EMT Madrid MobilityLabs as data source. Thank you and enjoy!", "aboutUses": "If you need to extend the daily use of this API, please, register your App in Mobilitylabs and use your own X-ClientId and  passKey instead of generic login (more info in https://mobilitylabs.emtmadrid.es/doc/new-app a

In [4]:
def get_stations():
    load_dotenv('./.env')
    token = os.environ.get("access_token")
    print(token)
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [5]:
get_stations()

70d9c557-201e-41c1-a674-f0e8953d0d22


{'code': '00',
 'description': '613 bases recovered',
 'datetime': '2024-02-02T14:09:50.531512',
 'data': [{'activate': 1,
   'address': 'Avenida del Ensanche de Vallecas, 9,',
   'dock_bikes': 9,
   'free_bases': 18,
   'geofence': {'type': 'Polygon',
    'coordinates': [[[-3.613119408, 40.370889958],
      [-3.613119408, 40.369989402],
      [-3.611941152, 40.369989402],
      [-3.611941152, 40.370889958],
      [-3.613119408, 40.370889958]]]},
   'geometry': {'type': 'Point', 'coordinates': [-3.61253028, 40.37043968]},
   'id': 2190,
   'integrator': '3c34ea72-2a03-4a3c-be97-cf32be4b0ebe',
   'light': 2,
   'name': '453 - Avenida del Ensanche de Vallecas, 9',
   'no_available': 0,
   'number': '453',
   'reservations_count': 0,
   'tipo_estacionPBSC': 'FIXED',
   'total_bases': 27,
   'virtualDelete': False,
   'virtual_bikes': [],
   'virtual_bikes_num': 0,
   'code_district': '18',
   'code_suburb': '183',
   'geofenced_capacity': 20,
   'bikesGo': []},
  {'activate': 1,
   'addre

In [24]:
stations = get_stations()
stations_real_time = pd.DataFrame(stations["data"])
stations_real_time.to_csv("bicimad_stations_backup.csv")


70d9c557-201e-41c1-a674-f0e8953d0d22


In [26]:
stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
stations_real_time = stations_real_time.drop(["geofence", "activate", "geometry", "integrator", "reservations_count", "no_available", "tipo_estacionPBSC", "virtualDelete", "virtual_bikes", "virtual_bikes_num", "code_suburb", "geofenced_capacity", "bikesGo"], axis=1)
stations_real_time['coordinates'] = list(zip(stations_real_time['longitude'], stations_real_time['latitude']))
stations_real_time

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
0,"Avenida del Ensanche de Vallecas, 9,",10,17,2190,2,"453 - Avenida del Ensanche de Vallecas, 9",453,27,18,-3.612530,40.370440,"(-3.61253028, 40.37043968)"
1,"Paseo de la Chopera,33,Comunidad de Madrid España",16,7,2205,2,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
2,"Paseo de la Castellana nº 164,Comunidad de Mad...",8,15,2225,2,157 - Castellana 164.,157,23,05,-3.689415,40.459137,"(-3.6894151, 40.4591366)"
3,"Guetaria , 84b,",12,12,2226,2,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168,"(-3.71569102, 40.36916772)"
4,"Calle de Gloria Fuertes, 181,",6,21,2151,0,"592 - Calle de Gloria Fuertes, 181",592,27,08,-3.724953,40.495697,"(-3.7249527, 40.4956969)"
...,...,...,...,...,...,...,...,...,...,...,...,...
608,"Calle Fernando el Católico nº 61,",16,11,1605,2,199 - Fernando el Católico,199,27,07,-3.714917,40.434361,"(-3.7149166, 40.4343611)"
609,"Calle San Bernardo nº 22,",9,10,1420,2,16 - San Bernardo,16,19,01,-3.707481,40.422937,"(-3.7074808485065, 40.4229374)"
610,"Calle Colombia nº 7,",3,21,1566,0,160 - Colombia,160,24,05,-3.676344,40.457247,"(-3.6763439, 40.4572466)"
611,"Calle Camino de los Vinateros nº 108,",2,16,1625,0,219 - Marroquina,219,18,14,-3.646734,40.408359,"(-3.6467338, 40.4083593)"


In [28]:
stations_real_time.to_csv("bicimad_backup.csv")

In [8]:
def get_district(df, district_number):
    result = df[df["code_district"] == district_number]
    return result

district_03 = get_district(stations_real_time, "03")
district_03

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
62,"Calle O'Donnell nº 28,",24,3,1465,1,61 - Narváez,61,27,03,-3.675314,40.421455,"(-3.6753139, 40.4214551)"
63,"Calle Ibiza nº 62,",5,18,1467,0,63 - Ibiza,63,23,03,-3.670896,40.417924,"(-3.6708959, 40.4179237)"
64,"Calle Antonio Maura nº 15,",22,2,1468,1,65 - Antonio Maura,65,24,03,-3.689425,40.416701,"(-3.689424797942769, 40.4167014469994)"
65,"Calle Espalter nº 1,",18,1,1470,1,68 - Espalter,68,19,03,-3.691323,40.412769,"(-3.6913232, 40.4127692)"
66,"Avenida de Alfonso XII nº 54,",13,14,1471,2,69 - Puerta del Ángel Caído,69,27,03,-3.688822,40.409808,"(-3.688822, 40.409808)"
67,"Avenida de Menéndez Pelayo nº 63,",23,4,1472,1,70 - Puerta del Doce de Octubre,70,27,03,-3.677873,40.415382,"(-3.677872542327878, 40.415382385574716)"
68,"Avenida de Nazaret nº 7,",8,15,1475,2,73 - Plaza de los Astros,73,23,03,-3.668700,40.411500,"(-3.6687, 40.4115)"
69,"Avenida de Menéndez Pelayo nº 69,",24,3,1476,1,74 - Puerta del Pacífico,74,27,03,-3.676589,40.412063,"(-3.676588763788984, 40.41206290978386)"
70,Avenida de Menéndez Pelayo con calle Poeta Est...,15,12,1477,2,75 - Menéndez Pelayo,75,27,03,-3.678475,40.408272,"(-3.6784749, 40.4082717)"
71,"Calle Doctor Esquerdo nº 191,",11,12,1487,2,84 - Valderribas,84,23,03,-3.672482,40.403572,"(-3.672481575393678, 40.40357156992851)"


In [9]:
district_01 = stations_real_time[stations_real_time["code_district"]== "01"]
district_02 = stations_real_time[stations_real_time["code_district"]== "02"]
district_03 = stations_real_time[stations_real_time["code_district"]== "03"]
district_04 = stations_real_time[stations_real_time["code_district"]== "04"]
district_05 = stations_real_time[stations_real_time["code_district"]== "05"]
district_06 = stations_real_time[stations_real_time["code_district"]== "06"]
district_07 = stations_real_time[stations_real_time["code_district"]== "07"]
district_08 = stations_real_time[stations_real_time["code_district"]== "08"]
district_09 = stations_real_time[stations_real_time["code_district"]== "09"]
district_10 = stations_real_time[stations_real_time["code_district"]== "10"]
district_11 = stations_real_time[stations_real_time["code_district"]== "11"]
district_12 = stations_real_time[stations_real_time["code_district"]== "12"]
district_13 = stations_real_time[stations_real_time["code_district"]== "13"]
district_14 = stations_real_time[stations_real_time["code_district"]== "14"]
district_15 = stations_real_time[stations_real_time["code_district"]== "15"]
district_16 = stations_real_time[stations_real_time["code_district"]== "16"]
district_17 = stations_real_time[stations_real_time["code_district"]== "17"]
district_18 = stations_real_time[stations_real_time["code_district"]== "18"]
district_19 = stations_real_time[stations_real_time["code_district"]== "19"]
district_20 = stations_real_time[stations_real_time["code_district"]== "20"]
district_21 = stations_real_time[stations_real_time["code_district"]== "21"]

In [10]:
district_02

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",18,5,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
57,"Calle Ribera de Curtidores nº 28,",15,12,1450,2,46 - Ribera de Curtidores,46,27,02,-3.707126,40.405315,"(-3.7071259, 40.4053153)"
98,Calle Teresa López Valcárcel esq. Teniente Cor...,2,22,2071,0,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
134,"Calle Delicias, 45,",12,7,2117,2,"273 - Calle Delicias, 45",273,19,02,-3.690918,40.403819,"(-3.69091831, 40.40381914)"
178,"Glorieta de Embajadores nº 6,",7,17,1451,0,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
200,"PASEO DE LOS MELANCÓLICOS 73,",7,16,2247,0,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,02,-3.718569,40.401782,"(-3.718569, 40.401782)"
213,"271 - Calle Amaltea , 1,",9,15,2265,2,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706,"(-3.68317, 40.394706)"
215,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",10,13,2267,2,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,23,02,-3.708212,40.402054,"(-3.7082123, 40.4020545)"
217,"Calle Méndez Alvaro, 44,",17,6,2269,1,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"
276,"Paseo de los Pontones, 45,",12,11,2299,2,"278 - Paseo de los Pontones, 45",278,23,02,-3.719600,40.404043,"(-3.7196, 40.4040433)"


In [11]:
def get_light1(df):
    df_light1 = df[df["light"]==1]
    return df_light1

In [12]:
get_light1(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
1,"Paseo de la Chopera,33,Comunidad de Madrid España",18,5,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
217,"Calle Méndez Alvaro, 44,",17,6,2269,1,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758,"(-3.68134346, 40.39675755)"
282,"Paseo virgen de puerto, 25,",15,0,2304,1,"277 - Paseo Virgen del Puerto, 25",277,15,02,-3.721178,40.411622,"(-3.721178, 40.411622)"
320,"Calle del tejo,",22,1,2305,1,272 - Calle del tejo,272,23,02,-3.679186,40.396630,"(-3.679186, 40.39663)"
345,"Calle Juan Martín el Empecinado nº 16,",23,0,1524,1,118 - Juan Martín,118,23,02,-3.688241,40.400781,"(-3.6882407, 40.400781)"
446,"Paseo de la Esperanza nº 2,",18,1,1569,1,163 - Paseo de la Esperanza,163,19,02,-3.706445,40.403759,"(-3.706445, 40.4037586)"
455,"Calle Jaime el Conquistador nº 30,",16,3,1581,1,175 - Jaime el Conquistador,175,19,02,-3.698345,40.396226,"(-3.69834452, 40.3962264)"
515,"Calle Ariel, 7,",17,7,2064,1,"269 - Calle Ariel, 7",269,24,02,-3.679702,40.393024,"(-3.67970176633988, 40.393023802369754)"
525,"Calle Toledo nº 181,",17,7,1575,1,169 - Pirámides,169,24,02,-3.713823,40.401589,"(-3.713823, 40.401589)"


In [13]:
def get_light0(df):
    df_light0 = df[df["light"]==0]
    return df_light0

In [14]:
get_light0(district_02)

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
98,Calle Teresa López Valcárcel esq. Teniente Cor...,2,22,2071,0,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629,"(-3.69328771, 40.38862908)"
178,"Glorieta de Embajadores nº 6,",7,17,1451,0,47 - Embajadores 1,47,24,02,-3.702800,40.404577,"(-3.7027998, 40.4045772)"
200,"PASEO DE LOS MELANCÓLICOS 73,",7,16,2247,0,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,02,-3.718569,40.401782,"(-3.718569, 40.401782)"
359,"Calle Sodio nº 1B,",6,17,1641,0,235 - Embajadores 191,235,23,02,-3.691076,40.389041,"(-3.691076, 40.389041)"
445,"Paseo de los Olmos nº 28,",4,18,1568,0,162 - Metro Pirámides,162,23,02,-3.710577,40.403497,"(-3.7105765, 40.4034968)"
454,"Paseo de Yeserías nº 45,",1,26,1580,0,174 - Madrid Río-Yeserías,174,27,02,-3.706526,40.397430,"(-3.7065258846557603, 40.39742958306358)"


In [15]:
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, vehicle_start).meters)
    return nearest_station

In [16]:
stations_real_time

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude,coordinates
0,"Avenida del Ensanche de Vallecas, 9,",9,18,2190,2,"453 - Avenida del Ensanche de Vallecas, 9",453,27,18,-3.612530,40.370440,"(-3.61253028, 40.37043968)"
1,"Paseo de la Chopera,33,Comunidad de Madrid España",18,5,2205,1,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000,"(-3.7003996647238746, 40.39500000000003)"
2,"Paseo de la Castellana nº 164,Comunidad de Mad...",11,12,2225,2,157 - Castellana 164.,157,23,05,-3.689415,40.459137,"(-3.6894151, 40.4591366)"
3,"Guetaria , 84b,",11,13,2226,2,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168,"(-3.71569102, 40.36916772)"
4,"Calle de Gloria Fuertes, 181,",6,21,2151,0,"592 - Calle de Gloria Fuertes, 181",592,27,08,-3.724953,40.495697,"(-3.7249527, 40.4956969)"
...,...,...,...,...,...,...,...,...,...,...,...,...
608,"Calle Fernando el Católico nº 61,",16,11,1605,2,199 - Fernando el Católico,199,27,07,-3.714917,40.434361,"(-3.7149166, 40.4343611)"
609,"Calle San Bernardo nº 22,",9,10,1420,2,16 - San Bernardo,16,19,01,-3.707481,40.422937,"(-3.7074808485065, 40.4229374)"
610,"Calle Colombia nº 7,",1,23,1566,0,160 - Colombia,160,24,05,-3.676344,40.457247,"(-3.6763439, 40.4572466)"
611,"Calle Camino de los Vinateros nº 108,",0,18,1625,0,219 - Marroquina,219,18,14,-3.646734,40.408359,"(-3.6467338, 40.4083593)"


In [17]:
load_dotenv('../.env')
client = ors.Client(key=os.environ.get("openroute_api_key"))

# Function to find the nearest station to given coordinates
def find_nearest_to_coords(df, coords):
    station_coordinates = df['coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, coords).meters)
    return nearest_station

# Function to create a route between two sets of coordinates
def create_route(client, start_coords, end_coords):
    return client.directions(
        coordinates=[start_coords, end_coords],
        profile='driving-car',
        format='geojson'
    )

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

# Creo un mapa con Folium
m = folium.Map(location=[vehicle_start[1], vehicle_start[0]], zoom_start=12)

# Añado marcador morado para la central (vehicle_start)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='CENTRAL EMT', icon=folium.Icon(color='purple')).add_to(m)

distrito02_low = get_light0(district_02).copy()
distrito02_high = get_light1(district_02).copy()

distrito02_high['visited'] = False
distrito02_low['visited'] = False

# Inicio de la ruta
current_coords = vehicle_start
van = "empty"

coords_list = [current_coords]

stop_counter = 1

for i in range(100):
    if van == "empty":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)].empty:
            nearest_station = find_nearest_to_coords(distrito02_high.loc[~distrito02_high['visited'] & (distrito02_high['light'] == 1)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'visited'] = True
            distrito02_high.loc[distrito02_high['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "full"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with high occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='orange')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
    elif van == "full":
        current_coords = coords_list[-1]
        # Verifica si quedan estaciones sin visitar
        if not distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)].empty:
            nearest_station = find_nearest_to_coords(distrito02_low.loc[~distrito02_low['visited'] & (distrito02_low['light'] == 0)], current_coords)
            coords_list.append(nearest_station)
            # Actualiza 'visited' y 'light' en el DataFrame original
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'visited'] = True
            distrito02_low.loc[distrito02_low['coordinates'] == nearest_station, 'light'] = 2
            route = create_route(client, coords_list[-2], coords_list[-1])
            van = "empty"
            folium.Marker(location=[nearest_station[1], nearest_station[0]],
                          popup=f"Station with low occupation\nNumber: {stop_counter}",
                          icon=folium.Icon(color='green')).add_to(m)
            stop_counter += 1
            folium.PolyLine(locations=[coord[::-1] for coord in route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)
final_route = create_route(client, coords_list[-1], vehicle_start)
folium.PolyLine(locations=[coord[::-1] for coord in final_route['features'][0]['geometry']['coordinates']],
                            color='red').add_to(m)

m